In [2]:
%load_ext autoreload
%autoreload 2
import risktools as rt
from pandas_datareader import data

In [5]:
rt.data.get_names()

['crudeOil',
 'cushing',
 'dflong',
 'dfwide',
 'eiaStocks',
 'eiaStorageCap',
 'eurodollar',
 'expiry_table',
 'fizdiffs',
 'futuresRef',
 'fxfwd',
 'holidaysOil',
 'ohlc',
 'planets',
 'refineryLPdata',
 'tickers_eia',
 'tradeCycle',
 'tradeHubs',
 'tradeprocess',
 'wti_swap',
 'stocks',
 'tsQuotes',
 'usSwapCurves',
 'usSwapCurvesPar']

In [31]:
rt.data.open_data('usSwapCurvesPar')

{'tradeDate': ['2023-08-18'], 'settleDate': ['2023-08-20'], 'dt': [0.0833], 'interpWhat': ['discount'], 'interpHow': ['spline']}


{'times': array([ 0.    ,  0.0833,  0.1667,  0.25  ,  0.3333,  0.4167,  0.5   ,
         0.5833,  0.6667,  0.75  ,  0.8333,  0.9167,  1.    ,  1.0833,
         1.1667,  1.25  ,  1.3333,  1.4167,  1.5   ,  1.5833,  1.6667,
         1.75  ,  1.8333,  1.9167,  2.    ,  2.0833,  2.1667,  2.25  ,
         2.3333,  2.4167,  2.5   ,  2.5833,  2.6667,  2.75  ,  2.8333,
         2.9167,  3.    ,  3.0833,  3.1667,  3.25  ,  3.3333,  3.4167,
         3.5   ,  3.5833,  3.6667,  3.75  ,  3.8333,  3.9167,  4.    ,
         4.0833,  4.1667,  4.25  ,  4.3333,  4.4167,  4.5   ,  4.5833,
         4.6667,  4.75  ,  4.8333,  4.9167,  5.    ,  5.0833,  5.1667,
         5.25  ,  5.3333,  5.4167,  5.5   ,  5.5833,  5.6667,  5.75  ,
         5.8333,  5.9167,  6.    ,  6.0833,  6.1667,  6.25  ,  6.3333,
         6.4167,  6.5   ,  6.5833,  6.6667,  6.75  ,  6.8333,  6.9167,
         7.    ,  7.0833,  7.1667,  7.25  ,  7.3333,  7.4167,  7.5   ,
         7.5833,  7.6667,  7.75  ,  7.8333,  7.9167,  8.    ,  8.083